In [1]:
using IJulia
installkernel("julia_ITensors","--sysimage=~/.julia/sysimages/sys_itensors.so")

[ Info: Installing julia_ITensors kernelspec in /Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9


"/Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9"

In [2]:
using ITensors
using ITensors.HDF5
using DataFrames

# Square Lattice

In [3]:
function square_lattice(Nx::Int, Ny::Int; yperiodic=false)::Lattice
  yperiodic = yperiodic && (Ny > 2)
  N = Nx * Ny
  Nbond = 2N - Ny + (yperiodic ? 0 : -Nx)
  latt = Lattice(undef, Nbond)
  b = 0
  for n in 1:N
    x = div(n - 1, Ny) + 1
    y = mod(n - 1, Ny) + 1
    if x < Nx
      latt[b += 1] = LatticeBond(n, n + Ny, x, y, x + 1, y)
    end
    if Ny > 1
      if y < Ny
        latt[b += 1] = LatticeBond(n, n + 1, x, y, x, y + 1)
      end
      if yperiodic && y == 1
        latt[b += 1] = LatticeBond(n, n + Ny - 1, x, y, x, y + Ny - 1)
      end
    end
  end
  return latt
end

square_lattice (generic function with 1 method)

# Triangular Lattice

In [4]:
function triangular_lattice(Nx::Int, Ny::Int; yperiodic=false)::Lattice
  yperiodic = yperiodic && (Ny > 2)
  N = Nx * Ny
  Nbond = 3N - 2Ny + (yperiodic ? 0 : -2Nx + 1)
  latt = Lattice(undef, Nbond)
  b = 0
  for n in 1:N
    x = div(n - 1, Ny) + 1
    y = mod(n - 1, Ny) + 1

    # x-direction bonds
    if x < Nx
      latt[b += 1] = LatticeBond(n, n + Ny)
    end

    # 2d bonds
    if Ny > 1
      # vertical / y-periodic diagonal bond
      if (n + 1 <= N) && ((y < Ny) || yperiodic)
        latt[b += 1] = LatticeBond(n, n + 1)
      end
      # periodic vertical bond
      if yperiodic && y == 1
        latt[b += 1] = LatticeBond(n, n + Ny - 1)
      end
      # diagonal bonds
      if x < Nx && y < Ny
        latt[b += 1] = LatticeBond(n, n + Ny + 1)
      end
    end
  end
  return latt
end

triangular_lattice (generic function with 1 method)

# TFIM

In [65]:
using ITensors

let
  Ny = 3
  Nx = 3

  N = Nx*Ny

  sites = siteinds("S=1/2", N)

  # Obtain an array of LatticeBond structs
  # which define nearest-neighbor site pairs
  # on the 2D square lattice (wrapped on a cylinder)
  lattice = square_lattice(Nx, Ny; yperiodic = false)

  # Define the Heisenberg spin Hamiltonian on this lattice
  ampo = OpSum()
  for b in lattice
    ampo .+=      4, "Sz", b.s1, "Sz", b.s2
  end
  for site in 1:N
    ampo .+=       4, "Sx", site
  end
  H = MPO(ampo,sites)

#   state = [isodd(n) ? "Up" : "Dn" for n=1:N]
  # Initialize wavefunction to a random MPS
  # of bond-dimension 10 with same quantum
  # numbers as `state`
  psi0 = randomMPS(sites,20)

  sweeps = Sweeps(10)
  maxdim!(sweeps,20,60,100,100,200,400,800)
  cutoff!(sweeps,1E-8)
  @show sweeps

  energy,psi = dmrg(H,psi0,sweeps)

  return
end

sweeps = Sweeps
1 cutoff=1.0E-08, maxdim=20, mindim=1, noise=0.0E+00
2 cutoff=1.0E-08, maxdim=60, mindim=1, noise=0.0E+00
3 cutoff=1.0E-08, maxdim=100, mindim=1, noise=0.0E+00
4 cutoff=1.0E-08, maxdim=100, mindim=1, noise=0.0E+00
5 cutoff=1.0E-08, maxdim=200, mindim=1, noise=0.0E+00
6 cutoff=1.0E-08, maxdim=400, mindim=1, noise=0.0E+00
7 cutoff=1.0E-08, maxdim=800, mindim=1, noise=0.0E+00
8 cutoff=1.0E-08, maxdim=800, mindim=1, noise=0.0E+00
9 cutoff=1.0E-08, maxdim=800, mindim=1, noise=0.0E+00
10 cutoff=1.0E-08, maxdim=800, mindim=1, noise=0.0E+00

After sweep 1 energy=-4.947517275594013  maxlinkdim=16 maxerr=8.33E-17 time=0.024
After sweep 2 energy=-4.948533946838914  maxlinkdim=14 maxerr=6.43E-09 time=0.010
After sweep 3 energy=-4.948533976272672  maxlinkdim=13 maxerr=8.53E-09 time=0.010
After sweep 4 energy=-4.94853397627401  maxlinkdim=13 maxerr=8.14E-09 time=0.010
After sweep 5 energy=-4.948533976274007  maxlinkdim=13 maxerr=8.14E-09 time=0.025
After sweep 6 energy=-4.94853397627

# Antiferromagnetic Heisenberg Model (Triangular Lattice!!!!)

In [7]:
using ITensors

let
  Ny = 6
  Nx = 6
  maxbd = 12
    
  N = Nx*Ny

  sites = siteinds("S=1/2", N;
                   conserve_qns = true)

  # Obtain an array of LatticeBond structs
  # which define nearest-neighbor site pairs
  # on the 2D TRIANGULAR(!) lattice 
  lattice = triangular_lattice(Nx, Ny; yperiodic = false)

  # Define the Heisenberg spin Hamiltonian on this lattice
  ampo = OpSum()
  for b in lattice
    ampo .+= 2, "S+", b.s1, "S-", b.s2
    ampo .+= 2, "S-", b.s1, "S+", b.s2
    ampo .+= 4,  "Sz", b.s1, "Sz", b.s2
  end
  H = MPO(ampo,sites)

  state = [isodd(n) ? "Up" : "Dn" for n=1:N]
  # Initialize wavefunction to a random MPS
  # of bond-dimension 10 with same quantum
  # numbers as `state`
  psi0 = randomMPS(sites,state,20)

  sweeps = Sweeps(10)
  maxdim!(sweeps,20,60,100,100,100,maxbd)
  cutoff!(sweeps,1E-8)
  @show sweeps

  energy,psi = dmrg(H,psi0,sweeps)
    
  @show energy/(4*Nx*Ny)

  return
end

sweeps = Sweeps
1 cutoff=1.0E-08, maxdim=20, mindim=1, noise=0.0E+00
2 cutoff=1.0E-08, maxdim=60, mindim=1, noise=0.0E+00
3 cutoff=1.0E-08, maxdim=100, mindim=1, noise=0.0E+00
4 cutoff=1.0E-08, maxdim=100, mindim=1, noise=0.0E+00
5 cutoff=1.0E-08, maxdim=100, mindim=1, noise=0.0E+00
6 cutoff=1.0E-08, maxdim=12, mindim=1, noise=0.0E+00
7 cutoff=1.0E-08, maxdim=12, mindim=1, noise=0.0E+00
8 cutoff=1.0E-08, maxdim=12, mindim=1, noise=0.0E+00
9 cutoff=1.0E-08, maxdim=12, mindim=1, noise=0.0E+00
10 cutoff=1.0E-08, maxdim=12, mindim=1, noise=0.0E+00

After sweep 1 energy=-67.63615195629286  maxlinkdim=20 maxerr=3.05E-03 time=0.481
After sweep 2 energy=-70.75733464787476  maxlinkdim=60 maxerr=1.29E-04 time=0.833
After sweep 3 energy=-71.43679636139738  maxlinkdim=100 maxerr=2.15E-04 time=1.925
After sweep 4 energy=-71.45875473992619  maxlinkdim=100 maxerr=3.09E-04 time=2.257
After sweep 5 energy=-71.46076862721954  maxlinkdim=100 maxerr=3.15E-04 time=2.244
After sweep 6 energy=-66.05101979232